<a href="https://colab.research.google.com/github/DataHiveMind/Fin_Engineering_and_AI_with_python/blob/main/Financial_Engineering_and_Statistical_ML_Analysis_with_Python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Objective:
- None

# Approach:
1. Loading Finanical Data then doing Data Preprocessing and Explorory Data Analysis(EDA)

2. Deep Diving into Time Series Analysis to understand a companies best time perfomance and its worst

3. Create Algorithmns to optimizie a portfolio and CAPM

4. Create Algortimic Tradin galgorithmns to find the most portfitable Strategies

5. Use Reinformaent Learning to use senitment analysis in stocks to see how poeple view each company

In [ ]:
import yfinance as yf
import os
import pandas as pd
import random
from glob import glob
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm

# Loading Financial data and EDA

In [ ]:
# Getting Financial Data from the Financial Data Source
Symbols  = ["MSFT", "AMZN", "GOOG", "IBM"]

if not os.path.exists('data'):
    os.mkdir('data')

for symbol in Symbols:
    if not os.path.exists(f"data/{symbol}.csv"):
        data = yf.download(symbol, start = "2010-01-01", end = "2018-12-31")
        if data.size > 0:
            data.to_csv(f"data/{symbol}.csv")
        else:
            print("Not Saving...")

for symbol in Symbols:
    s = open(f"data/{symbol}.csv").readlines()
    if len(s) < 10:
        os.system(f"rm data/{symbol}.csv")

[*********************100%***********************]  1 of 1 completed


In [ ]:
files = glob('data/*.csv')
full_df = None

for f in files:
    print(f)
    df = pd.read_csv(f)

    symbol = f.split('/')[1].split('.')[0]
    df['name'] = symbol

    if full_df is None:
        full_df = df
    else:
        full_df = full_df.append(df, ignore_index = True)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
df = pd.read_csv("/content/all_stocks_5yr.csv")
df.head()

In [ ]:
df['Name'].unique()

In [ ]:
df[df['Name'] == 'IBM'].head()

In [ ]:
ibm = df[df['Name'] == 'IBM']
ibm["close"].plot()

In [ ]:
df['date'].min(), df['date'].max()

In [ ]:
dates = pd.date_range(df['date'].min(), df['date'].max())
print(dates)

In [ ]:
# Changing dataframe for close price
close_price = pd.DataFrame(index = dates)
symbols = df['Name'].unique()

df2 = pd.DataFrame(data = ibm['close'].to_numpy(),
                   index=ibm['date'], columns = ['IBM'])
df2.head()

In [ ]:
# Fill in our close prices dataframe
for symbol in symbols:
  df_sym = df[df['Name'] == symbol]
  df_tmp = pd.DataFrame(data = df_sym['close'].to_numpy(),
                   index= df_sym['date'], columns = [symbol])
  close_price = close_price.join(df_tmp) # left-join by defualt

In [ ]:
close_price.head()

In [ ]:
!head sp500_close.csv

In [ ]:
close = pd.read_csv('sp500_close.csv', index_col = 0, parse_dates = True)
close.head()

In [ ]:
close['IBM'].plot()

In [ ]:
# How to Deal with Missing Data
close.dropna(axis = 0, how = "all", inplace = True)
close['IBM'].plot()

In [ ]:
close.isna().sum()

In [ ]:
# find any backwards filled stock data
close.iloc[0,:].isna().sum()
# fill NA values
close.fillna(method = 'ffill', inplace = True)

In [ ]:
close.isna().sum().sum()

In [ ]:
close.fillna(method = 'bfill', inplace = True)
close.isna().sum().sum()

In [ ]:
close.isna().sum().sum()

In [ ]:
close.plot(legend = False, figsize=(10,10))

In [ ]:
# normalized data
close_norm = close / close.iloc[0]
close_norm.plot(legend = False, figsize = (10,10))

In [ ]:
# Adjusted Close, Using sp500sub.csv if possible
df = pd.read_csv('sp500sub.csv', index_col = 'Date', parse_dates = True)
print(df.head())

goog = df[df["Name"] =='GOOG']
goog[['Close', 'Adj Close']].plot(figsize = (10,10))

aapl = df[df['Name'] == 'AAPL']
aapl[['Close', 'Adj Close']].plot(figsize = (10,10))

In [ ]:
# Returns
data = pd.read_csv('all_stocks_5yr.csv', parse_dates = True)
sbux = data[data["Name"]=='SBUX'].copy()
sbux.head()
sbux['close'].plot()

In [ ]:
# Calulate Return
sbux['prev_close'] = sbux['close'].shift(1)
sbux.head()

In [ ]:
sbux['return'] = sbux["close"]/sbux['prev_close']-1
sbux.head()

In [ ]:
sbux['return2'] = sbux['close'].pct_change(1)
sbux.head()

In [ ]:
sbux["return"].hist(bins = 100)
sbux['log_return'] =np.log(sbux['return']+1)
sbux.head()

In [ ]:
sbux['log_return'].hist(bins = 1000)
print("Mean of sbux: ", sbux['log_return'].mean())
print("Std of sbux: ", sbux['log_return'].std())

In [ ]:
# QQ-Plot
from scipy.stats import norm

x_list = np.linspace(sbux['return'].min(),
                     sbux['return'].max(),100)

y_list = norm.pdf(x_list,
                  loc = sbux['return'].mean(),
                  scale = sbux['return'].std())

plt.plot(x_list, y_list)
sbux['return'].hist(bins = 100, density = True)

In [ ]:
from scipy.stats import probplot

probplot(sbux['return'].dropna(),
         dist = 'norm',
         fit = True,
         plot = plt)

In [ ]:
import statsmodels.api as sm

sm.qqplot(sbux["return"].dropna(), line = 's')

In [ ]:
x_list = np.linspace(
    sbux["log_return"].min(),
    sbux["log_return"].max(),
    100
)

y_list = norm.pdf(x_list,
                  loc = sbux["log_return"].mean(),
                  scale = sbux["log_return"].std()
)

plt.plot(x_list, y_list)
sbux["log_return"].hist(bins = 100, density = True)

In [ ]:
sm.qqplot(sbux["log_return"].dropna(), line = 's')

In [ ]:
# T-Distribution
from scipy.stats import t
x_list = np.linspace(sbux['return'].min(),
                     sbux['return'].max(),100)
parms = t.fit(sbux['return'].dropna())
print(parms)

In [ ]:
df, loc, scale = parms
y_list = t.pdf(x_list, df, loc, scale)
plt.plot(x_list, y_list)
sbux['return'].hist(bins = 100, density = True)

In [ ]:
# Error will happen
sm.qqplot(sbux["return"].dropna(),
          dist = t,
          line = "s")

In [ ]:
#Error will happen
probplot(sbux["return"].dropna(),
         dist = t,
         fit = True,
         plot = plt)

In [ ]:
class myt:
  def __init__(self, df):
    self.df = df

  def fit(self, data):
    return x.fit(data)

  def ppf(self, x, loc = 0, scale = 1):
    return t.ppf(x, self.df, loc, scale)

In [ ]:
sm.qqplot(sbux["return"].dropna(),
          dist = myt(df),
          line = "s")

In [ ]:
probplot(sbux["return"].dropna(),
         dist = myt(df),
         fit = True,
         plot = plt)

In [ ]:
# Skewness and Kurtosis
"""
kurtosis = 3
kurtosis > 3: fat tails
kurtosis < 3: thin tails
"""

"""
skewness = 0
skewness > 0: right skewed
skewness < 0: left skewed
"""

sbux["return"].skew(), sbux["return"].kurtosis()

In [ ]:
# Conifdence Intrevals
"""
More Data -> More Accurate
Less Data -> Less Accurate
"""

value = sbux["return"].dropna().to_numpy()

m = value.mean()
s = value.std(ddof = 1)

low = m - 1.96 * s / np.sqrt(len(value))
high = m + 1.96 * s / np.sqrt(len(value))

sbux["return"].hist(bins = 100, density = True)
plt.axvline(m, label = "mean", color = 'red')
plt.axvline(low, label ="low",  color = 'green')
plt.axvline(high, label = "high", color = 'yellow')
plt.legend()

In [ ]:
plt.axvline(m, label = "mean", color = 'red')
plt.axvline(low, label ="low",  color = 'green')
plt.axvline(high, label = "high", color = 'yellow')
plt.axvline(0, label = "Zero", color = "blue")
plt.legend()

In [ ]:
# Statistical Testing
from scipy.stats import jarque_bera, normaltest

jb = jarque_bera(value)
print(jb)
print(normaltest(value))

In [ ]:
from scipy.stats import kstest

df, loc, scale = t.fit(value)
def cdf(x):
  return t.cdf(x, df, loc, scale)

ks = kstest(value, cdf)
print(ks)

In [ ]:
from scipy.stats import ttest_1samp

ttest_1samp(value, 0)

In [ ]:
# Covariance and Correlation
close = pd.read_csv('sp500_close.csv', index_col = 0, parse_dates = True)

symbols = ["AAPL", "IBM", "GOOG", "NFLX", "SBUX"]
subs = close[symbols].copy()
subs.dropna(axis = 0, how = "all", inplace = True)
subs.head()

In [ ]:
for symbol in symbols:
  subs[symbol + '_prev'] = subs[symbol].shift(1)
  subs[symbol + '_return'] = subs[symbol] / subs[symbol + '_prev'] - 1

rets = subs[[symbol +"_return" for symbol in symbols]].copy()
rets.head()

In [ ]:
import seaborn as sns
sns.pairplot(rets)

In [ ]:
rets.mean(), rets.std()

In [ ]:
plt.scatter(rets.std(), rets.mean())
plt.xlabel("Volatility")
plt.ylabel("Return")

In [ ]:
rets.corr(), rets.cov()

In [ ]:
# Alpha and Beta
"""
Alpha -> how much better/worse an asset Performs
Beta -> how much more/less volatile an asset is
"""
spy = pd.read_csv('SPY.csv', index_col = 0, parse_dates = True)
spy.head()

spy["SPY"] = spy["Close"].pct_change(1)

index = pd.read_csv("sp500sub.csv", index_col = 0, parse_dates = True)

aapl = index[index["Name"] == "AAPL"].copy()
aapl["AAPL"] = aapl["Close"].pct_change(1)

joined = aapl[["AAPL"]].join(spy["SPY"])
joined.head()

In [ ]:
joined.iloc[100:150].plot(figsize = (10,5))

In [ ]:
joined.plot.scatter(y = "AAPL", x = "SPY")

In [ ]:
# Make the dataset
joined.dropna(inplace = True)
x = joined[["AAPL"]].to_numpy()
y = joined[["SPY"]].to_numpy()

In [ ]:
from sklearn.linear_model import LinearRegression

model = LinearRegression()
model.fit(x,y)

x_predict = np.linspace(x.min(), x.max(), 5).reshape(-1,1)
y_predict = model.predict(x_predict)

plt.scatter(x,y)
plt.xlabel("SPY")
plt.ylabel("AAPL")
plt.plot(x_predict, y_predict)

In [ ]:
beta = model.coef_
alpha = model.intercept_
print("Beta Result: ", beta, "Alpha Result: ", alpha)

In [ ]:
# Manual Calculation for same graph
y_pred = beta * y_predict + alpha
plt.scatter(x,y)
plt.xlabel("SPY")
plt.ylabel("AAPL")
plt.plot(x_predict, y_pred)

In [ ]:
joined['AAPL'].std()/joined['SPY'].std()

In [ ]:
# Mixture of Gaussians
from sklearn.mixture import GaussianMixture

data = sbux['log_return'].dropna().to_numpy().reshape(-1,1)
model = GaussianMixture(n_components = 2)
model.fit(data)

weights = model.weights_
means = model.means_
cov = model.covariances_

print("Weights: ", weights)
print("Means: ", means)
print("Variances: ", cov)

In [ ]:
means = means.flatten()
var = cov.flatten()

In [ ]:
x_list = np.linspace(data.min(), data.max(), 100)
fx0 = norm.pdf(x_list, loc = means[0], scale = np.sqrt(var[0]))
fx1 = norm.pdf(x_list, loc = means[1], scale = np.sqrt(var[1]))
fx = weights[0] * fx0 + weights[1] * fx1

plt.plot(x_list, fx)
sbux['log_return'].hist(bins = 100, density = True)
plt.plot(x_list, fx, label = "mixture model")
plt.legend()

In [ ]:
# Volaitily Clustering
for i, symbol in enumerate(rets.columns):
  plt.subplot(len(rets.columns), 1, i + 1)
  plt.title(symbol)
  rets[symbol].plot(figsize = (12,18))

In [ ]:
# Price Simulation
p0 = sbux.iloc[-1]["close"]
price = [p0]
returns = sbux["return"].dropna()

for _ in range(100):
  p = price[-1] * (1 + np.random.choice(returns))
  price.append(p)

plt.plot(price)

In [ ]:
df, loc, scale = t.fit(sbux["return"].dropna())
p0 = sbux.iloc[-1]["close"]
price = [p0]

for _ in range(100):
  r = t.rvs(df, loc, scale)
  p = price[-1] * (1 + r)
  price.append(p)

plt.plot(price)

# Time Series Analysis